In [61]:
# Se importan todas las librerías necesarias
import pandas as pd
import random
import time
import numpy as np
import smtplib, os
from email.message import EmailMessage
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders
from dotenv import load_dotenv
load_dotenv()
from datetime import datetime

In [62]:
# Se cargan datos simulados desde los csv correspondientes
clientes = pd.read_csv('../datasets/csv_simulados_full/clientes.csv')
categorias = pd.read_csv('../datasets/csv_simulados_full/categorias.csv')
cliente_categoria = pd.read_csv('../datasets/csv_simulados_full/cliente_categoria.csv')
auditorias = pd.read_csv('../datasets/csv_simulados_full/auditorias.csv')
fotos_cargadas = pd.read_csv('../datasets/csv_simulados_full/fotos_cargadas.csv')

In [63]:
# Tabla de cliente, 30 registros
clientes

,cliente_id,cliente_nombre
0,1,Cliente_1
1,2,Cliente_2
2,3,Cliente_3
3,4,Cliente_4
4,5,Cliente_5
5,6,Cliente_6
6,7,Cliente_7
7,8,Cliente_8
8,9,Cliente_9
9,10,Cliente_10


In [64]:
# Tabla de categorías, 15 registros
categorias

,categoria_id,categoria_nombre
0,1,Bebidas
1,2,Lácteos
2,3,Snacks
3,4,Carnes
4,5,Limpieza
5,6,Frutas
6,7,Verduras
7,8,Panadería
8,9,Tecnología
9,10,Electrodomésticos


In [65]:
# Relacion cliente - categoría
cliente_categoria

,cliente_id,categoria_id
0,1,5
1,1,2
2,2,4
3,2,8
4,3,3
...,...,...
119,29,3
120,30,4
121,30,14
122,30,13


In [66]:
auditorias

,auditoria_id,fecha,tienda,auditor,cliente_id
0,1,2025-08-07 23:09:32,Tienda_36,Auditor_20,24
1,2,2025-02-21 05:49:17,Tienda_34,Auditor_1,30
2,3,2025-02-10 07:41:06,Tienda_18,Auditor_6,20
3,4,2025-01-20 05:19:57,Tienda_8,Auditor_9,4
4,5,2025-08-17 07:29:57,Tienda_26,Auditor_13,12
...,...,...,...,...,...
495,496,2025-05-29 06:21:44,Tienda_29,Auditor_5,25
496,497,2025-03-09 09:00:50,Tienda_44,Auditor_15,23
497,498,2025-07-25 16:31:06,Tienda_2,Auditor_10,30
498,499,2025-02-13 09:33:04,Tienda_25,Auditor_18,17


In [67]:
auditorias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   auditoria_id  500 non-null    int64 
 1   fecha         500 non-null    object
 2   tienda        500 non-null    object
 3   auditor       500 non-null    object
 4   cliente_id    500 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 19.7+ KB


In [68]:
fotos_cargadas

,foto_id,auditoria_id,categoria_id,tipo
0,1,1,NaN,control_inicio
1,2,1,NaN,control_fin
2,3,1,3.0,categoria
3,4,1,5.0,categoria
4,5,1,15.0,categoria
...,...,...,...,...
2921,2922,500,11.0,categoria
2922,2923,500,8.0,categoria
2923,2924,500,13.0,categoria
2924,2925,500,14.0,categoria


In [69]:
fotos_cargadas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2926 entries, 0 to 2925
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   foto_id       2926 non-null   int64  
 1   auditoria_id  2926 non-null   int64  
 2   categoria_id  1926 non-null   float64
 3   tipo          2926 non-null   object 
dtypes: float64(1), int64(2), object(1)
memory usage: 91.6+ KB


# **Funciones de automatización (proceso)**

In [70]:
# Función para detectar alertas
def detectar_alertas(auditorias, fotos, cliente_categoria, clientes):
    """
    Detecta auditorías con número de fotos insuficiente.
    
    Retorna un DataFrame con:
    - auditoria_id
    - cliente
    - fotos_esperadas
    - fotos_cargadas
    - fotos_faltantes
    """
    
    alertas = []
    
    for _, aud in auditorias.iterrows():
        auditoria_id = aud["auditoria_id"]
        cliente_id = aud["cliente_id"]
        
        # Nombre del cliente
        cliente = clientes.loc[clientes["cliente_id"] == cliente_id, "cliente_nombre"].values[0]
        
        # Fotos de la auditoría
        fotos_aud = fotos[fotos["auditoria_id"] == auditoria_id]
        fotos_cargadas = len(fotos_aud)
        
        # Categorías que debe auditar este cliente
        categorias_cliente = cliente_categoria[cliente_categoria["cliente_id"] == cliente_id]
        fotos_esperadas = len(categorias_cliente) + 2  # +2 controles
        
        # Calcular faltantes
        fotos_faltantes = max(0, fotos_esperadas - fotos_cargadas)
        
        # Si faltan fotos → generar alerta
        if fotos_faltantes > 0:
            alertas.append({
                "auditoria_id": auditoria_id,
                "cliente": cliente,
                "fotos_esperadas": fotos_esperadas,
                "fotos_cargadas": fotos_cargadas,
                "fotos_faltantes": fotos_faltantes
            })
    
    return pd.DataFrame(alertas)


In [71]:
# Función para exportar el df resultante a excel
def exportar_excel(df, filename="../datasets/generados/alertas.xlsx"):
    if not df.empty:
        df.to_excel(filename, index=False)
        print(f"✅ Reporte generado: {filename}")
    else:
        print("🎉 No se encontraron alertas, todo está completo.")


In [72]:
def enviar_excel_por_correo(alertas, auditorias, destinatario, asunto="Alertas Auditorías"): 
    EMAIL = os.environ.get("ALERT_EMAIL")
    PASS = os.environ.get("ALERT_PASS")
    
    if not EMAIL or not PASS:
        raise ValueError("Configura ALERT_EMAIL y ALERT_PASS en tu archivo .env")

    msg = EmailMessage()
    msg["Subject"] = asunto
    msg["From"] = EMAIL
    msg["To"] = destinatario
    msg.set_content("Muy buen día, adjunto a este correo encontrará los reportes de alertas (completo y del día).")

    # --- Reporte completo ---
    ruta_completo = "../datasets/generados/alertas_completo.xlsx"
    alertas.to_excel(ruta_completo, index=False)

    with open(ruta_completo, "rb") as f:
        msg.add_attachment(
            f.read(),
            maintype="application",
            subtype="vnd.openxmlformats-officedocument.spreadsheetml.sheet",
            filename=os.path.basename(ruta_completo)
        )

    # --- Reporte de hoy ---
    hoy = datetime.now().date()
    alertas_hoy = alertas.merge(
        auditorias[["auditoria_id", "fecha"]],
        on="auditoria_id",
        how="left"
    )

    # Convertir 'fecha' a datetime para evitar error con .dt
    alertas_hoy["fecha"] = pd.to_datetime(alertas_hoy["fecha"], errors="coerce")
    alertas_hoy = alertas_hoy[alertas_hoy["fecha"].dt.date == hoy]
    alertas_hoy = alertas_hoy.drop(columns=["fecha"])

    ruta_hoy = "../datasets/generados/alertas_hoy.xlsx"
    alertas_hoy.to_excel(ruta_hoy, index=False)

    with open(ruta_hoy, "rb") as f:
        msg.add_attachment(
            f.read(),
            maintype="application",
            subtype="vnd.openxmlformats-officedocument.spreadsheetml.sheet",
            filename=os.path.basename(ruta_hoy)
        )

    # Enviar correo
    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
        smtp.login(EMAIL, PASS)
        smtp.send_message(msg)

    print(f"✅ Correo enviado a {destinatario} con reportes completo y diario")




# **Funciones de automatización (simulación)**

In [73]:
def nueva_auditoria(auditoria_id, cliente_id, cliente_categoria, start_foto_id, forzar_error=False):
    """
    Genera una auditoría nueva con sus fotos asociadas.
    Retorna:
      - DataFrame con la auditoría
      - DataFrame con las fotos cargadas
      - último id de foto usado
    """
    # 1. Crear el registro de auditoría
    auditoria = pd.DataFrame([{
        "auditoria_id": auditoria_id,
        "fecha": pd.Timestamp.now(),
        "tienda": f"Tienda_{random.randint(1, 50)}",
        "auditor": f"Auditor_{random.randint(1, 20)}",
        "cliente_id": cliente_id
    }])

    # 2. Categorías del cliente
    categorias = cliente_categoria[cliente_categoria["cliente_id"] == cliente_id]["categoria_id"].tolist()
    fotos_min = len(categorias) + 2  
    
    # 3. Cuántas fotos incluir
    if forzar_error and fotos_min > 2:
        total_fotos = random.randint(2, fotos_min - 1)
    else:
        total_fotos = fotos_min

    fotos = []
    foto_id = start_foto_id  # arrancamos desde el último disponible

    # 4. Fotos de control
    fotos.append({"foto_id": int(foto_id), "auditoria_id": auditoria_id, "categoria_id": np.nan, "tipo": "control_inicio"})
    foto_id += 1
    fotos.append({"foto_id": int(foto_id), "auditoria_id": auditoria_id, "categoria_id": np.nan, "tipo": "control_fin"})
    foto_id += 1

    # 5. Fotos de categorías
    categorias_seleccionadas = random.sample(categorias, min(len(categorias), total_fotos - 2))
    for cat in categorias_seleccionadas:
        fotos.append({"foto_id": int(foto_id), "auditoria_id": auditoria_id, "categoria_id": cat, "tipo": "categoria"})
        foto_id += 1

    # 6. Retornar todo
    return auditoria, pd.DataFrame(fotos), foto_id - 1  # último id usado


In [74]:
def ciclo_simulacion(
    auditorias, fotos_cargadas, cliente_categoria, clientes,
    intervalo=60,                  
    destinatario="destinatario@ejemplo.com",
    ciclos=5                       
):
    """
    Simula el flujo de:
    - Generar nuevas auditorías (opcional, aleatorio por ciclo).
    - Detectar alertas.
    - Exportar auditorías simuladas.
    - Enviar Excel con reportes (completo y diario).
    """
    # Para controlar IDs
    next_auditoria_id = auditorias["auditoria_id"].max() + 1
    next_foto_id = fotos_cargadas["foto_id"].max() + 1

    contador = 0
    while ciclos is None or contador < ciclos:
        print(f"\n⏳ Ciclo {contador+1} iniciado...")

        # Elegir aleatoriamente si generar nuevas auditorías
        if contador == 0:
            generar_nuevas_ciclo = False 
        elif contador == 1:
            generar_nuevas_ciclo = True
        elif contador >= 2:     
            generar_nuevas_ciclo = random.choice([True, False])

        n_nuevas_ciclo = random.choice([20, 30, 40, 60, 80, 100, 160, 320]) if generar_nuevas_ciclo else 0 # 5, 10, 20, 40, 80, 160, 

        # 1. Generar nuevas auditorías
        if generar_nuevas_ciclo:
            auditorias_nuevas = []
            fotos_nuevas = []
            for i in range(n_nuevas_ciclo):
                cliente_id = random.randint(1, clientes["cliente_id"].max())
                forzar_error = random.choice([True, False])

                aud, fotos_df, next_foto_id = nueva_auditoria(
                    next_auditoria_id, cliente_id, cliente_categoria,
                    start_foto_id=next_foto_id, forzar_error=forzar_error
                )
                auditorias_nuevas.append(aud)
                fotos_nuevas.append(fotos_df)
                next_auditoria_id += 1

            auditorias = pd.concat([auditorias] + auditorias_nuevas, ignore_index=True)
            auditorias['fecha'] = pd.to_datetime(auditorias["fecha"], errors="coerce")
            fotos_cargadas = pd.concat([fotos_cargadas] + fotos_nuevas, ignore_index=True)
            print(f"🆕 Se generaron {n_nuevas_ciclo} auditorías nuevas.")
        else:
            print("ℹ️ No se generaron auditorías nuevas en este ciclo.")

        # 2. Detectar alertas
        alertas = detectar_alertas(auditorias, fotos_cargadas, cliente_categoria, clientes)

        # 3. Exportar auditorías simuladas
        exportar_excel(auditorias, "../datasets/generados/auditorias_sim.xlsx")
        exportar_excel(fotos_cargadas, "../datasets/generados/fotos_cargados_sim.xlsx")

        # 4. Enviar correos (completo + diario)
        enviar_excel_por_correo(alertas, auditorias, destinatario)

        # 5. Esperar hasta el siguiente ciclo
        contador += 1
        print(f"✅ Ciclo {contador} terminado. Esperando {intervalo} segundos...\n")
        time.sleep(intervalo)

    return auditorias, fotos_cargadas


# **Simulación**

In [86]:
# Simulación de X ciclos, X segundoS entre cada uno, primer ciclo sin nuevas auditorías
auditorias_sim, fotos_cargadas_sim = ciclo_simulacion(
    auditorias, fotos_cargadas, cliente_categoria, clientes,
    intervalo=1,      # En segundos
    destinatario="gabriel.garcia@utp.edu.co",
    ciclos=1    # correr 3 veces y detener
)



⏳ Ciclo 1 iniciado...
ℹ️ No se generaron auditorías nuevas en este ciclo.
✅ Reporte generado: ../datasets/generados/auditorias_sim.xlsx
✅ Reporte generado: ../datasets/generados/fotos_cargados_sim.xlsx
✅ Correo enviado a gabriel.garcia@utp.edu.co con reportes completo y diario
✅ Ciclo 1 terminado. Esperando 1 segundos...



In [76]:
alertas_completo = pd.read_excel('../datasets/generados/alertas_completo.xlsx')
alertas_hoy = pd.read_excel('../datasets/generados/alertas_hoy.xlsx')

In [77]:
alertas_completo

,auditoria_id,cliente,fotos_esperadas,fotos_cargadas,fotos_faltantes
0,4,Cliente_4,6,2,4
1,6,Cliente_17,4,2,2
2,13,Cliente_8,9,4,5
3,14,Cliente_25,6,2,4
4,20,Cliente_29,6,3,3
...,...,...,...,...,...
141,669,Cliente_30,6,5,1
142,671,Cliente_28,6,3,3
143,674,Cliente_4,6,2,4
144,676,Cliente_23,8,6,2


In [78]:
alertas_completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   auditoria_id     146 non-null    int64 
 1   cliente          146 non-null    object
 2   fotos_esperadas  146 non-null    int64 
 3   fotos_cargadas   146 non-null    int64 
 4   fotos_faltantes  146 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 5.8+ KB


In [79]:
alertas_hoy

,auditoria_id,cliente,fotos_esperadas,fotos_cargadas,fotos_faltantes
0,503,Cliente_30,6,2,4
1,504,Cliente_6,5,4,1
2,506,Cliente_25,6,4,2
3,507,Cliente_16,7,3,4
4,508,Cliente_13,7,4,3
...,...,...,...,...,...
76,669,Cliente_30,6,5,1
77,671,Cliente_28,6,3,3
78,674,Cliente_4,6,2,4
79,676,Cliente_23,8,6,2


In [80]:
alertas_hoy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   auditoria_id     81 non-null     int64 
 1   cliente          81 non-null     object
 2   fotos_esperadas  81 non-null     int64 
 3   fotos_cargadas   81 non-null     int64 
 4   fotos_faltantes  81 non-null     int64 
dtypes: int64(4), object(1)
memory usage: 3.3+ KB


In [81]:
auditorias_sim

,auditoria_id,fecha,tienda,auditor,cliente_id
0,1,2025-08-07 23:09:32.000000,Tienda_36,Auditor_20,24
1,2,2025-02-21 05:49:17.000000,Tienda_34,Auditor_1,30
2,3,2025-02-10 07:41:06.000000,Tienda_18,Auditor_6,20
3,4,2025-01-20 05:19:57.000000,Tienda_8,Auditor_9,4
4,5,2025-08-17 07:29:57.000000,Tienda_26,Auditor_13,12
...,...,...,...,...,...
675,676,2025-09-08 11:17:18.735016,Tienda_35,Auditor_15,23
676,677,2025-09-08 11:17:18.735016,Tienda_40,Auditor_15,16
677,678,2025-09-08 11:17:18.735016,Tienda_22,Auditor_4,21
678,679,2025-09-08 11:17:18.735016,Tienda_39,Auditor_10,1


In [82]:
auditorias_sim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680 entries, 0 to 679
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   auditoria_id  680 non-null    int64         
 1   fecha         680 non-null    datetime64[ns]
 2   tienda        680 non-null    object        
 3   auditor       680 non-null    object        
 4   cliente_id    680 non-null    int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 26.7+ KB


In [83]:
fotos_cargadas_sim

,foto_id,auditoria_id,categoria_id,tipo
0,1,1,NaN,control_inicio
1,2,1,NaN,control_fin
2,3,1,3.0,categoria
3,4,1,5.0,categoria
4,5,1,15.0,categoria
...,...,...,...,...
3811,3634,679,5.0,categoria
3812,3634,680,NaN,control_inicio
3813,3635,680,NaN,control_fin
3814,3636,680,13.0,categoria


In [84]:
fotos_cargadas_sim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3816 entries, 0 to 3815
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   foto_id       3816 non-null   int64  
 1   auditoria_id  3816 non-null   int64  
 2   categoria_id  2456 non-null   float64
 3   tipo          3816 non-null   object 
dtypes: float64(1), int64(2), object(1)
memory usage: 119.4+ KB


In [85]:
# Pausar ejecución hasta que el usuario siga manualmente
import sys
sys.exit(0)

SystemExit: 0

c:\Users\gabri\OneDrive\Escritorio\proyectos\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# **power automate**

In [ ]:
import time
import pyautogui
from pywinauto import Application

pbix_file = r"C:\Users\gabri\OneDrive\Escritorio\proyectos\auditory_market\dashboards\dahsboard.pbix"
pbidesktop_path = r"C:\Program Files\Microsoft Power BI Desktop\bin\PBIDesktop.exe"

print("🚀 Abriendo Power BI...")
app = Application(backend="uia").start(f'"{pbidesktop_path}" "{pbix_file}"')
main = app.window(title_re=r".*dahsboard.*|.*\.pbix.*")
main.wait('visible', timeout=180)
main.set_focus()
time.sleep(3)

# Forzar foco con click # Esta linea no se usa 
rect = main.rectangle()
pyautogui.click(rect.left + 50, rect.top + 50)
time.sleep(6)

# ALT → activar KeyTips
print("🎹 ALT...")
pyautogui.press("alt")
time.sleep(2)

# A → abrir Archivo
print("🎹 A (Archivo)...")
pyautogui.press("a")
time.sleep(1.3)

# Enter → entrar al menú Archivo
print("🎹 Enter (entrar al menú Archivo)...")
pyautogui.press("enter")
time.sleep(0.9)

# 7 veces ↓ → Exportar
print("🎹 7 veces ↓ (hasta Exportar)...")
pyautogui.press("down", presses=7, interval=0.4)
time.sleep(0.3)

# Enter → abrir Exportar
print("🎹 Enter (abrir Exportar)...")
pyautogui.press("enter")
time.sleep(0.3)

# ↓ → Exportar a PDF
print("🎹 ↓ (seleccionar Exportar a PDF)...")
pyautogui.press("down")
time.sleep(0.3)

# Enter → abrir PDF en Chrome
print("🎹 Enter (abrir Exportar a PDF)...")
pyautogui.press("enter")
time.sleep(8)

# --- CTRL+S en Chrome para abrir el diálogo Guardar ---
print("🖫 En Chrome: CTRL + S...")
time.sleep(1)          # pequeño respiro para que Chrome termine de cargar el PDF
pyautogui.hotkey("ctrl", "s")
time.sleep(1)          # espera a que aparezca el cuadro "Guardar como"

# --- Escribir la ruta completa del archivo en el cuadro Guardar como ---
pdf_file = r"C:\Users\gabri\OneDrive\Escritorio\proyectos\auditory_market\dashboards\guardados\dashboard.pdf"

print(f"💾 Guardando en: {pdf_file}")
time.sleep(0.3)

# Escribir ruta completa
pyautogui.typewrite(pdf_file)
time.sleep(0.3)

# Enter para confirmar guardado
pyautogui.press("enter")
time.sleep(0.3)

# --- Confirmar reemplazo si aparece popup ---
time.sleep(0.2)  # darle un respiro a Chrome para mostrar el aviso
print("⚠️ Si aparece confirmación de reemplazo: mover a 'Sí' y presionar Enter...")
pyautogui.press("left")   # mover selección a "Sí"
time.sleep(0.2)
pyautogui.press("enter")  # confirmar

# --- Cerrar Power BI con la X ---
print("🛑 Cerrando Power BI...")
try:
    main.close()  # intenta cerrar con el botón X
    time.sleep(2)
    print("✅ Power BI cerrado con la X.")
except Exception as e:
    print("⚠️ No se pudo cerrar con la X, forzando kill:", e)
    app.kill()







🚀 Abriendo Power BI...
🎹 ALT...
🎹 A (Archivo)...
🎹 Enter (entrar al menú Archivo)...
🎹 7 veces ↓ (hasta Exportar)...
🎹 Enter (abrir Exportar)...
🎹 ↓ (seleccionar Exportar a PDF)...
🎹 Enter (abrir Exportar a PDF)...
🖫 En Chrome: CTRL + S...
💾 Guardando en: C:\Users\gabri\OneDrive\Escritorio\proyectos\auditory_market\dashboards\guardados\dashboard.pdf
⚠️ Si aparece confirmación de reemplazo: mover a 'Sí' y presionar Enter...
🛑 Cerrando Power BI...
✅ Power BI cerrado con la X.


In [ ]:
EMAIL_USER = os.getenv("ALERT_EMAIL")   # tu correo remitente
EMAIL_PASS = os.getenv("ALERT_PASS")   # tu contraseña / app password
EMAIL_TO   = 'gabriel.garcia@utp.edu.co'     # destinatario (o varios separados por coma)

# Ruta del PDF (el que ya generaste con tu script)
pdf_file = r"C:\Users\gabri\OneDrive\Escritorio\proyectos\auditory_market\dashboards\guardados\dashboard.pdf"

def enviar_informe():
    msg = MIMEMultipart()
    msg["From"] = EMAIL_USER
    msg["To"] = EMAIL_TO
    msg["Subject"] = "📊 Informe automático - Dashboard actualizado"

    # Cuerpo del correo
    body = "Hola,\n\nAdjunto encontrarás el informe actualizado en PDF.\n\nSaludos."
    msg.attach(MIMEText(body, "plain"))


    # 📎 Adjuntar archivo PDF
    with open(pdf_file, "rb") as f:  # Abrir el PDF en modo binario (lectura de bytes)
        part = MIMEBase("application", "octet-stream")  # Crear contenedor MIME para archivo binario genérico
        part.set_payload(f.read())  # Cargar el contenido del PDF en el contenedor
        encoders.encode_base64(part)  # Codificar en base64 (SMTP solo acepta texto ASCII)
        # Agregar encabezado indicando que es un adjunto y su nombre de archivo
        part.add_header("Content-Disposition", f'attachment; filename="{os.path.basename(pdf_file)}"')
        msg.attach(part)  # Adjuntar el PDF al mensaje principal (MIMEMultipart)

    # 📤 Enviar correo (ejemplo con Gmail)
    try:
        server = smtplib.SMTP("smtp.gmail.com", 587)  # Conectar al servidor SMTP de Gmail en puerto 587
        server.starttls()  # Iniciar conexión segura (TLS)
        server.login(EMAIL_USER, EMAIL_PASS)  # Autenticarse con usuario y contraseña/token
        # Enviar correo: remitente, lista de destinatarios y mensaje en formato cadena
        server.sendmail(EMAIL_USER, EMAIL_TO.split(","), msg.as_string())
        server.quit()  # Cerrar conexión con el servidor
        print("✅ Correo enviado correctamente.")
    except Exception as e:
        print("❌ Error al enviar correo:", e)  # Capturar y mostrar error si algo falla

# ▶ Ejecutar la función de envío
enviar_informe()


✅ Correo enviado correctamente.
